# 用t5-base做纠错

In [1]:
from happytransformer import HappyTextToText, TTSettings

happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")

args = TTSettings(num_beams=5, min_length=1)

# Add the prefix "grammar: " before each input 
result = happy_tt.generate_text("grammar: This sentences has has bads grammar.", args=args)

print(result.text) # This sentence has bad grammar.


/home/hyxu/anaconda3/envs/cor/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
05/15/2023 15:33:02 - INFO - happytransformer.happy_transformer -   Using model: cuda


This sentence has bad grammar.


In [4]:
import os
path = "./data/"
with open(os.path.join(path, "pre30w-en-ur.en")) as f:
    pre_data = f.readlines()
len(pre_data)

255697

In [5]:
pre_data[:10]

["Those who believed, and those who emigrated and fought in the way of Allah, those have hope of Allah's mercy, and Allah is Forgiving, Merciful.\n",
 'They ask you about wine and gambling. Tell them, "There is great sin in them and some benefit for people. But their sinfulness outweighs their benefit. They ask you what they should give. Tell them, "Let it be whatever you want." Thus God makes clear to you His revelations that you may give thought.\n',
 "And in this world and in the Hereafter. And they ask you concerning orphans. Say: 'It is better to set things right between them. If you mix with them, then they are your brethren. Allah knows the corrupter from the set-upright. Had Allah willed, He would have put you to hardship. Allah is All-Mighty, All-Wise.'\n",
 'And marry not the idolatresses until they believe; and a believing maid is better than a believing woman, though she please you; and marry not the idolaters until they believe; and a believing servant is better than a bel

In [11]:
from tqdm import tqdm
total_result = []
for s in tqdm(pre_data[:40]):
    res = happy_tt.generate_text(f"grammar: {s}", args=args)
    total_result.append(res.text)

NameError: name 'pre_data' is not defined

In [10]:
total_result

["Those who believed, and those who emigrated and fought in the way of Allah, those have hope of Allah's mercy, and Allah is Forgiving, Merciful.",
 'They ask you about wine and gambling. Tell them, "There is great sin in them and some benefit for people, but their sinfulness outweighs their benefit." Thus God makes clear to you His revelations that you may give thought',
 "And in this world and in the Hereafter. And they ask you concerning orphans. Say: 'It is better to set things right between them. If you mix with them, then they are your brethren.",
 'And marry not the idolatresses until they believe; and a believing maid is better than a believing woman, though she pleases you; and marry not the idolaters until they believe; these invite unto the',
 'They ask you about women\'s menses. Tell them: "It is a period of stress. So keep away from women during it until they are clean. When they are clean, you may go to them as God has ',
 'And if they decide upon divorce, then surely All

In [5]:
from tqdm import tqdm
def process_result(path):
    dir_lst = os.listdir(path)
    results = []
    dir_lst.sort()
    for d in tqdm(dir_lst):
        if "thread" not in d: continue
        id = int(d[len('thread'):])

        res_path = os.path.join(path, d, "res.txt")
        with open(res_path, 'r') as f:
            res = f.readlines()
        results += res
    with open(os.path.join(path, "30w-t50-en-ur.en"), 'w') as f:
        f.writelines(results)

In [6]:
import os
path = "./data"
dir_lst = os.listdir(path)
dir_lst

['thread1',
 '30w-t50-en-ur.en',
 'thread6',
 'thread5',
 'thread7',
 'thread4',
 'thread0',
 't5-base',
 'thread2',
 'en-ur',
 'pre30w-en-ur.en',
 'thread3']

In [15]:
dir_lst.sort()
dir_lst

['30w-t50-en-ur.en',
 'en-ur',
 'pre30w-en-ur.en',
 'thread0',
 'thread1',
 'thread2',
 'thread3',
 'thread4',
 'thread5',
 'thread6',
 'thread7',
 'thread8',
 'thread9']

In [7]:
process_result(path)

100%|██████████| 12/12 [00:00<00:00, 127.22it/s]


# 换用large 测试

In [23]:
import re
from tqdm.auto import tqdm
#@title define functions
#@markdown - this defines the `split_text` and `correct_text` functions
#@markdown - **NOTE:** you may need to adjust the regex in `split_text` depending on your source text

def split_text(text: str) -> list:
    """
    Split a string of text into a list of sentence batches.

    Parameters:
    text (str): The text to be split into sentence batches.

    Returns:
    list: A list of sentence batches. Each sentence batch is a list of sentences.
    """
    # Split the text into sentences using regex
    sentences = re.split(r"(?<=[^A-Z].[.?]) +(?=[A-Z])", text)

    # Initialize a list to store the sentence batches
    sentence_batches = []

    # Initialize a temporary list to store the current batch of sentences
    temp_batch = []

    # Iterate through the sentences
    for sentence in sentences:
        # Add the sentence to the temporary batch
        temp_batch.append(sentence)

        # If the length of the temporary batch is between 2 and 3 sentences, or if it is the last batch, add it to the list of sentence batches
        if len(temp_batch) >= 2 and len(temp_batch) <= 3 or sentence == sentences[-1]:
            sentence_batches.append(temp_batch)
            temp_batch = []

    return sentence_batches


def correct_text(text, checker, corrector, separator: str = " ") -> str:
    """
    Correct the grammar in a string of text using a text-classification and text-generation pipeline.

    Parameters:
    text (str): The text to be corrected.
    checker (transformers.pipeline.Pipeline): The text-classification pipeline to use for checking the grammar quality of the text.
    corrector (transformers.pipeline.Pipeline): The text-generation pipeline to use for correcting the text.
    separator (str, optional): The separator to use when joining the corrected text into a single string. Default is a space character.

    Returns:
    str: The corrected text.
    """
    # Split the text into sentence batches
    # sentence_batches = split_text(text)
    sentence_batches = text

    # Initialize a list to store the corrected text
    corrected_text = []

    # Iterate through the sentence batches
    for batch in tqdm(
        sentence_batches, total=len(sentence_batches), desc="correcting text.."
    ):
        # Join the sentences in the batch into a single string
        raw_text = " ".join(batch)

        # Check the grammar quality of the text using the text-classification pipeline
        results = checker(raw_text)

        # Only correct the text if the results of the text-classification are not LABEL_1 or are LABEL_1 with a score below 0.9
        if results[0]["label"] != "LABEL_1" or (
            results[0]["label"] == "LABEL_1" and results[0]["score"] < 0.9
        ):
            # Correct the text using the text-generation pipeline
            corrected_batch = corrector(raw_text)
            corrected_text.append(corrected_batch[0]["generated_text"])
        else:
            corrected_text.append(raw_text)

    # Join the corrected text into a single string
    corrected_text = separator.join(corrected_text)

    return corrected_text

In [7]:
import os
import torch
from transformers import pipeline
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Initialize the text-classification pipeline

cache_dir = "../cache_dir/"

# Initialize the text-generation pipeline
corrector = pipeline(
        "text2text-generation",
        "pszemraj/flan-t5-large-grammar-synthesis",
        device=torch.device("cuda:0")
    )



In [9]:
corrected_batch = corrector(["I saw a frog sal his child, wow my god!", "we"])

In [10]:
corrected_batch

[{'generated_text': 'I saw a frog chasing a child, wow!'},
 {'generated_text': 'We are all here for a reason.'}]

In [11]:
with open("./data/pre30w-en-ur.en") as f:
    pre_data = f.readlines()
len(pre_data)

255697

In [13]:
from tqdm import tqdm

In [15]:
batch_size = 64
total_num = len(pre_data) // batch_size if len(pre_data) % batch_size == 0 else len(pre_data) // batch_size +1
total_result = []
for i in tqdm(range(total_num)):
    batch = pre_data[i*batch_size : (i+1)*batch_size]
    res = corrector(batch)
    res = [r["generated_text"] for r in res]
    total_result += res
len(total_result)

  0%|          | 1/3996 [03:49<255:12:37, 229.98s/it]


KeyboardInterrupt: 

In [1]:
import evaluate
def compute_chrf(predictions, references, word_order=2):
    metric = evaluate.load('chrf')
    res = metric.compute(predictions=predictions, references=references, word_order=word_order)
    return res

/home/hyxu/anaconda3/envs/cor/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open("./predictions.txt") as f:
    preds = f.readlines()
with open("./references.txt") as f:
    refer = f.readlines()
preds = [p.rstrip("\n") for p in preds]
refer = [[p.rstrip("\n")] for p in refer]

In [4]:
compute_chrf(preds, refer)

{'score': 43.21665375738997, 'char_order': 6, 'word_order': 2, 'beta': 2}